In [36]:
import pandas as pd
import numpy as np
import ipaddress
import logging
from sklearn.preprocessing import StandardScaler, LabelEncoder
from datetime import datetime

In [37]:
# 🔹 Set up logging
logging.basicConfig(level=logging.INFO, format="%(asctime)s - %(levelname)s - %(message)s")
logger = logging.getLogger(__name__)

In [38]:
# 🔹 Load datasets
def load_data():
    try:
        fraud_data = pd.read_csv("/content/drive/MyDrive/weak_8_data/Data-20250205T200552Z-001/Data/Fraud_Data.csv")
        ip_country_data = pd.read_csv("/content/drive/MyDrive/weak_8_data/Data-20250205T200552Z-001/Data/IpAddress_to_Country.csv")
        #credit_data=pd.read_csv("/content/drive/MyDrive/weak_8_data/Data-20250205T200552Z-001/Data/IpAddress_to_Country.csv")
        logger.info(f"Loaded Fraud_Data.csv with shape {fraud_data.shape}")
        logger.info(f"Loaded IpAddress_to_Country.csv with shape {ip_country_data.shape}")
        #logger.info(f"Loaded Credit data with shape {credit_data.shape}")
        return fraud_data, ip_country_data # credit_data
    except Exception as e:
        logger.error(f"Error loading datasets: {e}")
        raise

In [39]:
# 🔹 Handle missing values
def handle_missing_values(df):
    missing_before = df.isnull().sum()
    df.fillna({
        "browser": "Unknown",
        "source": "Unknown",
        "purchase_value": df["purchase_value"].median(),
    }, inplace=True)
    logger.info(f"Handled missing values. Before: {missing_before.sum()}, After: {df.isnull().sum().sum()}")
    return df

In [40]:
# 🔹 Data cleaning: remove duplicates, correct types
def clean_data(df):
    before = df.shape[0]
    df.drop_duplicates(inplace=True)
    df["purchase_value"] = pd.to_numeric(df["purchase_value"], errors="coerce")
    df["signup_time"] = pd.to_datetime(df["signup_time"], errors="coerce")
    df["purchase_time"] = pd.to_datetime(df["purchase_time"], errors="coerce")
    logger.info(f"Removed {before - df.shape[0]} duplicate rows.")
    return df

In [41]:
# 🔹 Merge IP address data with a proper range-based lookup
def merge_geolocation(fraud_df, ip_df):
    try:
        # Convert columns to numeric safely
        fraud_df["ip_address"] = pd.to_numeric(fraud_df["ip_address"], errors="coerce")
        ip_df["lower_bound_ip_address"] = pd.to_numeric(ip_df["lower_bound_ip_address"], errors="coerce")
        ip_df["upper_bound_ip_address"] = pd.to_numeric(ip_df["upper_bound_ip_address"], errors="coerce")

        # Drop NaNs to avoid errors
        fraud_df.dropna(subset=["ip_address"], inplace=True)
        ip_df.dropna(subset=["lower_bound_ip_address", "upper_bound_ip_address"], inplace=True)

        # Use integer type if possible
        fraud_df["ip_address"] = fraud_df["ip_address"].astype("int")
        ip_df["lower_bound_ip_address"] = ip_df["lower_bound_ip_address"].astype("int")
        ip_df["upper_bound_ip_address"] = ip_df["upper_bound_ip_address"].astype("int")

        # Perform range-based lookup
        def find_country(ip):
            match = ip_df[(ip_df["lower_bound_ip_address"] <= ip) & (ip_df["upper_bound_ip_address"] >= ip)]
            return match["country"].values[0] if not match.empty else "Unknown"

        fraud_df["country"] = fraud_df["ip_address"].apply(find_country)

        logger.info(f"Merged geolocation data successfully. New shape: {fraud_df.shape}")
    except Exception as e:
        logger.error(f"Error in merging geolocation data: {e}")

    return fraud_df



In [42]:
def feature_engineering(df):
    try:
        # Ensure purchase_time is datetime
        df["purchase_time"] = pd.to_datetime(df["purchase_time"], errors="coerce")

        # Remove rows with missing timestamps
        df.dropna(subset=["purchase_time"], inplace=True)

        # Calculate transaction velocity (time difference between transactions)
        df["transaction_velocity"] = df.groupby("user_id")["purchase_time"].diff().dt.total_seconds()

        # Fix: Assign back to df explicitly to avoid FutureWarning
        df["transaction_velocity"] = df["transaction_velocity"].fillna(86400)

        # Calculate transaction frequency (count of transactions per user)
        df["transaction_frequency"] = df.groupby("user_id")["user_id"].transform("count")

        # Extract time-based features
        df["hour_of_day"] = df["purchase_time"].dt.hour
        df["day_of_week"] = df["purchase_time"].dt.weekday

        logger.info("Created new features: transaction velocity, transaction frequency, hour_of_day, day_of_week")
    except Exception as e:
        logger.error(f"Error in feature engineering: {e}")

    return df


In [43]:
# 🔹 Normalize & Encode
def normalize_encode(df):
    try:
        if df.shape[0] == 0:
            logger.warning("No data available for normalization.")
            return df

        scaler = StandardScaler()
        df["purchase_value"] = scaler.fit_transform(df[["purchase_value"]])

        encoder = LabelEncoder()
        df["browser"] = encoder.fit_transform(df["browser"])
        df["source"] = encoder.fit_transform(df["source"])
        df["sex"] = encoder.fit_transform(df["sex"])

        logger.info("Normalized purchase_value and encoded categorical features.")
    except Exception as e:
        logger.error(f"Error in normalization and encoding: {e}")
    return df

In [44]:
# 🔹 Full Pipeline Execution
def fraud_detection_pipeline():
    fraud_data, ip_country_data = load_data()
    fraud_data = handle_missing_values(fraud_data)
    fraud_data = clean_data(fraud_data)
    fraud_data = merge_geolocation(fraud_data, ip_country_data)
    fraud_data = feature_engineering(fraud_data)
    fraud_data = normalize_encode(fraud_data)

    logger.info(f"Final dataset shape: {fraud_data.shape}")
    return fraud_data


In [45]:
# 🚀 Run the pipeline
if __name__ == "__main__":
    final_df = fraud_detection_pipeline()
    final_df.to_csv("Preprocessed_Fraud_Data.csv", index=False)
    logger.info("Saved preprocessed data to Preprocessed_Fraud_Data.csv")

In [46]:
print(final_df.head())

   user_id         signup_time       purchase_time  purchase_value  \
0    22058 2015-02-24 22:55:49 2015-04-18 02:47:11       -0.160204   
1   333320 2015-06-07 20:39:50 2015-06-08 01:38:54       -1.142592   
2     1359 2015-01-01 18:52:44 2015-01-01 18:52:45       -1.197169   
3   150084 2015-04-28 21:13:25 2015-05-04 13:54:50        0.385567   
4   221365 2015-07-21 07:09:52 2015-09-09 18:40:53        0.112681   

       device_id  source  browser  sex  age  ip_address  class        country  \
0  QVPSPJUOCKZAR       2        0    1   39   732758368      0          Japan   
1  EOGFQPIZPYXFZ       0        0    0   53   350311387      0  United States   
2  YSSKYOSJHPPLJ       2        3    1   53  2621473820      1  United States   
3  ATGTXKYKUDUQN       2        4    1   41  3840542443      0        Unknown   
4  NAUITBZFJKHWW       0        4    1   45   415583117      0  United States   

   transaction_velocity  transaction_frequency  hour_of_day  day_of_week  
0               8